<a href="https://colab.research.google.com/github/Hamid-Ghalandari/Apoptosis/blob/main/Obesity_in_NHANES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
crude_dataset = pd.read_excel('/content/drive/MyDrive/2017-prepand NHANES datasets/Relevant Dataset_New.xlsx')
crude_dataset

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,DMDBORN4,DMDEDUC2,DMDMARTZ,RIDEXPRG,INDFMPIR,...,DR2T_D_TOTAL,DR2T_D_MILK,DR2T_D_YOGURT,DR2T_D_CHEESE,DR2T_OILS,DR2T_SOLID_FATS,DR2T_ADD_SUGARS,DR2T_A_DRINKS,DR1TSODI,DR2TSODI
0,109266,2,29,5,6,2,5.0,3.0,2.0,5.00,...,2.54,0.08,0.29,2.17,30.83,25.44,1.51,0.00,2001.0,3439.0
1,109267,2,21,2,2,2,4.0,3.0,3.0,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109268,2,18,3,3,1,NaN,NaN,NaN,1.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109271,1,49,3,3,1,2.0,3.0,NaN,NaN,...,1.98,0.35,0.00,1.63,49.54,115.34,30.97,0.00,5446.0,6157.0
4,109273,1,36,3,3,1,4.0,3.0,NaN,0.83,...,4.07,4.07,0.00,0.00,32.03,51.88,11.84,0.00,1990.0,2949.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9688,124815,1,52,4,4,1,4.0,2.0,NaN,5.00,...,0.00,0.00,0.00,0.00,1.60,1.89,1.09,2.02,3415.0,1541.0
9689,124817,2,67,1,1,2,2.0,1.0,NaN,2.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1129.0,NaN
9690,124818,1,40,4,4,1,5.0,1.0,NaN,3.82,...,3.12,0.55,0.00,2.57,28.76,67.08,46.32,0.00,7742.0,6120.0
9691,124821,1,63,4,4,1,2.0,2.0,NaN,3.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4718.0,NaN


In [3]:
# crude_dataset.columns.to_list()

In [4]:
# crude_dataset['RIDAGEYR'].describe()
# crude_dataset['DPQ100'].isna().sum()

List of unnecessary objects(need to be dropped after the preprocessing is complete):
RIDEXPRG (Pregnancy satus)**
BMDSTATS (Body Measures Component Status Code)
BMXWT (Weight (kg))
BMIWT (Weight Comment)**
BMXHT (Standing Height (cm))
BMIHT (Standing Height Comment)**
BMDBMIC (BMI Category - Children/Youth)
BPXOSY1 (Systolic - 1st oscillometric reading)
BPXODI1 (Diastolic - 1st oscillometric reading)
BPXOSY2 (Systolic - 2nd oscillometric reading)
BPXODI2 (Diastolic - 2nd oscillometric reading)
BPXOSY3 (Systolic - 3rd oscillometric reading)
BPXODI3 (Diastolic - 3rd oscillometric reading)
DPQs (DPQ010 - 100)**
DR1DRSTZ (Dietary recall status(day1))**
DR2DRSTZ (Dietary recall status(day2))**
**bold text**!!! DO NOT FORGET to also remove the related dietary data of day 1 if only the dietary data of the first day is used!!!
ECD010 (Mother's age when born)
ECD070B (Weight at birth, ounces)
ECQ080 (Weight more/less than 5.5 lbs)
ECQ090 (Weight more/less than 9.0 lbs)
MCQ160B (Ever told had congestive heart failure)**
MCQ170M (Do you still have thyroid problem) **bold text** Studies seem inconsisitent in showing having a thyroid problem is associated with disturbed anthropometric measurements. Moreover, the 'problems' are not specified.
MCQ170L (Do you still have a liver condition)



Future outcomes:
BMXWAIST (Waist Circumference (cm))
BMIWAIST (Waist Circumference Comment)
**# Be SUPER careful to exclude cases from which data could not be obtained (when regarded as outcome)**
BMXHIP (Hip Circumference (cm))
BMIHIP (Hip Circumference Comment)
**# Be SUPER careful to exclude cases from which data could not be obtained (when regarded as outcome)**




** Used as a exclusion criteria/to calculate a score before deleting the attribute


In [5]:
# Defining a function to remove indices (rows) based on an exclusing criterion

def exclusion_criteria(df, index_excl_codes):
    modified_df = df.copy()
    for column_name, exclusion_criteria in index_excl_codes:
        try:
            modified_df = modified_df[~df[column_name].isin(exclusion_criteria)]
        except KeyError:
            print(f'Column {column_name} not found in the dataset.')
    return modified_df



In [6]:
def save_dataset_excel(df,file_path, index = False):
  try:
    df.to_excel(file_path, index=index)
    print(f'Successfully saved to {file_path}')
  except Exception as e:
    print(f'error{str(e)}')


In [7]:
# Defining the exclusion criteria

index_excl_codes = [
    ('RIDEXPRG' , [1]), # Removing the pregnant subjects
    ('BMIWT', [1,4]),   # Excluding subjects with code=1 (weight not obtained) and code=4 (subjects' weights were measured when a 'medical appliance' was attached to  them) of BMIWT
                        # Note: Preliminary analyses showed that having a medical appliance (code = 4) seem to overestimate the BMI measurements
    ('BMIHT', [1,3]),  # Excluding subjects with code=1 (height not obtained) and code=3 (not straight) of BMIHT
    ('DPQ010', [7,9]), # Excluding subjects who refused to answer (code = 7) or replied 'don't know' (code = 9) to PHQ-9 questionnaire (depression)
    ('DPQ020', [7,9]),
    ('DPQ030', [7,9]),
    ('DPQ040', [7,9]),
    ('DPQ050', [7,9]),
    ('DPQ060', [7,9]),
    ('DPQ070', [7,9]),
    ('DPQ080', [7,9]),
    ('DPQ090', [7,9]),
    ('DPQ100', [7,9]),
    ('DR1DRSTZ', [2,5]), # Excluding subjects with code=2 (unreliable dietary data) and code=5 (dietary data was not obtained(days 1 and 2))
    ('DR2DRSTZ', [2,5]),  # Due to remarkable number of fallouts, two sets of datasets were created (one which includes valid subjects who completed day 1 of
                          # of dietary questionnaire and another one which includes the subjects with valid information for both days).
    ('MCQ160B', [1]), # Excluding subjects with a history of congestive heart failure
]

In [8]:
# Removing the cases having the exclusion criteria and producing the modified dataset

df_modified_excl = exclusion_criteria(crude_dataset, index_excl_codes)
df_modified_excl

<ipython-input-5-44c44bc55911>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  modified_df = modified_df[~df[column_name].isin(exclusion_criteria)]


,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,DMDBORN4,DMDEDUC2,DMDMARTZ,RIDEXPRG,INDFMPIR,...,DR2T_D_TOTAL,DR2T_D_MILK,DR2T_D_YOGURT,DR2T_D_CHEESE,DR2T_OILS,DR2T_SOLID_FATS,DR2T_ADD_SUGARS,DR2T_A_DRINKS,DR1TSODI,DR2TSODI
0,109266,2,29,5,6,2,5.0,3.0,2.0,5.00,...,2.54,0.08,0.29,2.17,30.83,25.44,1.51,0.00,2001.0,3439.0
1,109267,2,21,2,2,2,4.0,3.0,3.0,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109268,2,18,3,3,1,NaN,NaN,NaN,1.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109271,1,49,3,3,1,2.0,3.0,NaN,NaN,...,1.98,0.35,0.00,1.63,49.54,115.34,30.97,0.00,5446.0,6157.0
4,109273,1,36,3,3,1,4.0,3.0,NaN,0.83,...,4.07,4.07,0.00,0.00,32.03,51.88,11.84,0.00,1990.0,2949.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9684,124811,1,76,3,3,1,5.0,1.0,NaN,5.00,...,1.05,1.00,0.00,0.00,102.76,34.96,18.04,0.00,4678.0,4469.0
9685,124812,2,62,2,2,1,4.0,1.0,NaN,5.00,...,0.55,0.55,0.00,0.00,0.00,50.91,10.40,0.25,3543.0,1531.0
9686,124813,2,43,4,4,2,5.0,1.0,2.0,NaN,...,0.00,0.00,0.00,0.00,17.44,0.83,0.00,0.00,3807.0,2462.0
9687,124814,1,64,4,4,1,3.0,2.0,NaN,2.00,...,1.34,0.04,0.00,1.30,10.67,36.00,28.19,3.55,1756.0,2149.0


In [9]:
### Defining a function that will return 'NaN' whenever the denominator is zero (to prevent the production od infinite numbers when dividing two numbers)
def safe_division(numerator, denominator):
  if denominator == 0:
    return np.nan
  else:
    return numerator / denominator



In [10]:
#%% Exploring the variables


# df_modified_excl['RIDEXPRG'].describe()
# df_modified_excl['BMIWT'].describe()
# df_modified_excl['BMIHT'].describe()
# df_modified_excl['DPQ100'].describe()
# df_modified_excl['DPQ100'].isnull().sum()
# df_modified_excl['PAD615'].isnull().sum()
# count_1 = (df_modified_excl['MCQ160B']==1).sum()
# count_1


In [11]:
# Calculating the depression score (PHQ-9), Physical activity components
#%%% The codes are adjusted so that missing values are not summed up #%%%

depression_vars = ['DPQ010','DPQ020','DPQ030','DPQ040','DPQ050','DPQ060','DPQ070','DPQ080','DPQ090','DPQ100']
df_modified_excl['Depression_total'] = np.where(df_modified_excl[depression_vars].notna().all(axis=1), df_modified_excl[depression_vars].sum(axis=1), None)
PA_vigorous_total_vars = ['PAD615', 'PAD660'] # Duration of vigorous physical activity (Work + Recreational)
df_modified_excl['PA_vigorous_total'] = np.where(df_modified_excl[PA_vigorous_total_vars].notna().all(axis=1), df_modified_excl[PA_vigorous_total_vars].sum(axis=1), None)



In [12]:
#%% Calculating some components needed for HEI-2020

## To refernce the method used:
## https://epi.grants.cancer.gov/hei/hei-scoring-method.html#:~:text=To%20convert%20to%20a%20percent%20of%20calories%20basis%2C,to%20kcal%2C%20prior%20to%20dividing%20by%20total%20energy.
##!! 'Whole fruit' is calculated differently from NCI method; however, the results were verified and are identical

# Whole Fruit
Whole_Fruit_D1 = (df_modified_excl['DR1T_F_TOTAL'] - df_modified_excl['DR1T_F_JUICE']) # Total fruit minus fruit juices (Day1) (cup eq.)
df_modified_excl['Whole_Fruit_D1'] = Whole_Fruit_D1
Whole_Fruit_D2 = (df_modified_excl['DR2T_F_TOTAL'] - df_modified_excl['DR2T_F_JUICE'])
df_modified_excl['Whole_Fruit_D2'] = Whole_Fruit_D2

# Total vegetables
Total_Veg_incl_legumes_D1 = df_modified_excl['DR1T_V_TOTAL'] + df_modified_excl['DR1T_V_LEGUMES'] # Total vegetables calculated as FPED total veg. plus legumes counted as vegetables (Day1) (cup eq.)
df_modified_excl['Total_Veg_incl_legumes_D1'] = Total_Veg_incl_legumes_D1
Total_Veg_incl_legumes_D2 = df_modified_excl['DR2T_V_TOTAL'] + df_modified_excl['DR2T_V_LEGUMES']
df_modified_excl['Total_Veg_incl_legumes_D2'] = Total_Veg_incl_legumes_D2

# Dark greens and beans
Dark_Greens_and_Beans_D1 =  df_modified_excl['DR1T_V_DRKGR'] + df_modified_excl['DR1T_V_LEGUMES'] # Dark Green vegetables plus legumes counted as vegetables (Day1) (cup eq.)
df_modified_excl['Dark_Greens_and_Beans_D1'] = Dark_Greens_and_Beans_D1
Dark_Greens_and_Beans_D2 =  df_modified_excl['DR2T_V_DRKGR'] + df_modified_excl['DR2T_V_LEGUMES']
df_modified_excl['Dark_Greens_and_Beans_D2'] = Dark_Greens_and_Beans_D2

# Total protein
Total_Protein_D1 = df_modified_excl['DR1T_PF_TOTAL'] + df_modified_excl['DR1T_PF_LEGUMES'] # Total meat, poultry, seafood, organ meats, cured meat, eggs, soy, and nuts and seeds; excludes legumes (oz. eq.)
# + Legumes computed as protein foods (oz. eq.)
df_modified_excl['Total_Protein_D1'] = Total_Protein_D1
Total_Protein_D2 = df_modified_excl['DR2T_PF_TOTAL'] + df_modified_excl['DR2T_PF_LEGUMES']
df_modified_excl['Total_Protein_D2'] = Total_Protein_D2

# Seafood and plant proteins
Seafood_and_plant_proteins_D1 = df_modified_excl['DR1T_PF_SEAFD_HI'] + df_modified_excl['DR1T_PF_SEAFD_LOW'] + df_modified_excl['DR1T_PF_SOY'] + df_modified_excl['DR1T_PF_NUTSDS'] + df_modified_excl['DR1T_PF_LEGUMES']
# Seafood (finfish, shellfish and other seafood) high in n-3 fatty acids (oz. eq.) + Seafood (finfish, shellfish and other seafood) low in n-3 fatty acids (oz. eq.)
# + Soy products, excluding calcium fortified soy milk and immature soybeans (oz. eq.) + Peanuts, tree nuts, and seeds, excludes coconut (oz. eq.)
# + Legumes computed as protein foods (oz. eq.)
df_modified_excl ['Seafood_and_plant_proteins_D1'] = Seafood_and_plant_proteins_D1
Seafood_and_plant_proteins_D2 = df_modified_excl['DR2T_PF_SEAFD_HI'] + df_modified_excl['DR2T_PF_SEAFD_LOW'] + df_modified_excl['DR2T_PF_SOY'] + df_modified_excl['DR2T_PF_NUTSDS'] + df_modified_excl['DR2T_PF_LEGUMES']
df_modified_excl ['Seafood_and_plant_proteins_D2'] = Seafood_and_plant_proteins_D2



# Fatty acids
PUFA_MUFA_D1 = (df_modified_excl['DR1TPFAT'] + df_modified_excl['DR1TMFAT']) # PUFA + MUFA (d1)
df_modified_excl['PUFA_MUFA_D1'] = PUFA_MUFA_D1
# print(df_modified_excl['PUFA_MUFA_D1'].describe())
df_modified_excl['Fatty_acids_D1'] = df_modified_excl.apply(lambda row: safe_division(row['PUFA_MUFA_D1'], row['DR1TSFAT']), axis=1).round(2)
print(df_modified_excl['Fatty_acids_D1'].describe())
PUFA_MUFA_D2 = (df_modified_excl['DR2TPFAT'] + df_modified_excl['DR2TMFAT']) # PUFA + MUFA (d2)
df_modified_excl['PUFA_MUFA_D2'] = PUFA_MUFA_D2
# print(df_modified_excl['PUFA_MUFA_D2'].describe())
df_modified_excl['Fatty_acids_D2'] = df_modified_excl.apply(lambda row: safe_division(row['PUFA_MUFA_D2'], row['DR2TSFAT']), axis=1).round(2)
print(df_modified_excl['Fatty_acids_D2'].describe())





# Sodium (in grams)
Sodium_D1 = df_modified_excl['DR1TSODI'] / 1000
df_modified_excl['Sodium_D1'] = Sodium_D1
Sodium_D2 = df_modified_excl['DR2TSODI'] / 1000
df_modified_excl['Sodium_D2'] = Sodium_D2

modified_df_hei = df_modified_excl.copy()
modified_df_hei





count    6540.000000
mean        2.003891
std         0.814138
min         0.150000
25%         1.430000
50%         1.850000
75%         2.400000
max         8.020000
Name: Fatty_acids_D1, dtype: float64
count    6539.000000
mean        2.044320
std         0.886362
min         0.440000
25%         1.440000
50%         1.860000
75%         2.470000
max        10.450000
Name: Fatty_acids_D2, dtype: float64


,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,DMDBORN4,DMDEDUC2,DMDMARTZ,RIDEXPRG,INDFMPIR,...,Total_Protein_D1,Total_Protein_D2,Seafood_and_plant_proteins_D1,Seafood_and_plant_proteins_D2,PUFA_MUFA_D1,Fatty_acids_D1,PUFA_MUFA_D2,Fatty_acids_D2,Sodium_D1,Sodium_D2
0,109266,2,29,5,6,2,5.0,3.0,2.0,5.00,...,2.59,1.83,2.41,0.11,44.684,2.02,32.790,1.48,2.001,3.439
1,109267,2,21,2,2,2,4.0,3.0,3.0,5.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109268,2,18,3,3,1,NaN,NaN,NaN,1.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109271,1,49,3,3,1,2.0,3.0,NaN,NaN,...,7.67,11.89,0.00,0.07,59.432,1.52,121.981,1.80,5.446,6.157
4,109273,1,36,3,3,1,4.0,3.0,NaN,0.83,...,3.69,4.45,0.00,0.00,8.010,2.97,55.645,1.39,1.990,2.949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9684,124811,1,76,3,3,1,5.0,1.0,NaN,5.00,...,9.43,9.80,5.06,4.23,57.340,0.73,117.469,2.88,4.678,4.469
9685,124812,2,62,2,2,1,4.0,1.0,NaN,5.00,...,3.54,2.79,0.00,0.00,77.360,2.19,21.434,0.64,3.543,1.531
9686,124813,2,43,4,4,2,5.0,1.0,2.0,NaN,...,6.93,7.69,3.58,6.99,42.450,2.31,18.106,4.20,3.807,2.462
9687,124814,1,64,4,4,1,3.0,2.0,NaN,2.00,...,1.63,3.76,0.91,0.60,40.176,2.28,27.867,1.25,1.756,2.149


In [13]:
#%% Calculating the energy-adjusted (in 1000 Cal energy intake) components



# Total Fruit
Total_fruit_EA_d1 = ((modified_df_hei['DR1T_F_TOTAL'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Total_fruit_EA_d1'] = Total_fruit_EA_d1
Total_fruit_EA_d2 = ((modified_df_hei['DR2T_F_TOTAL'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Total_fruit_EA_d2'] = Total_fruit_EA_d2


# Whole Fruit
Whole_fruit_EA_d1 = ((modified_df_hei['Whole_Fruit_D1'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Whole_fruit_EA_d1'] = Whole_fruit_EA_d1
Whole_fruit_EA_d2 = ((modified_df_hei['Whole_Fruit_D2'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Whole_fruit_EA_d2'] = Whole_fruit_EA_d2

# Total Vegetables
Total_veg_EA_d1 = ((modified_df_hei['Total_Veg_incl_legumes_D1'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Total_veg_EA_d1'] = Total_veg_EA_d1
Total_veg_EA_d2 = ((modified_df_hei['Total_Veg_incl_legumes_D2'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Total_veg_EA_d2'] = Total_veg_EA_d2

# Greans and Beans
Greens_Beans_EA_d1 = ((modified_df_hei['Dark_Greens_and_Beans_D1'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Greens_Beans_EA_d1'] = Greens_Beans_EA_d1
Greens_Beans_EA_d2 = ((modified_df_hei['Dark_Greens_and_Beans_D2'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Greens_Beans_EA_d2'] = Greens_Beans_EA_d2

# Whole Grains
Whole_grains_EA_d1 = ((modified_df_hei['DR1T_G_WHOLE'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Whole_grains_EA_d1'] = Whole_grains_EA_d1
Whole_grains_EA_d2 = ((modified_df_hei['DR2T_G_WHOLE'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Whole_grains_EA_d2'] = Whole_grains_EA_d2

# Dairy
Dairy_EA_d1 = ((modified_df_hei['DR1T_D_TOTAL'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Dairy_EA_d1'] = Dairy_EA_d1
Dairy_EA_d2 = ((modified_df_hei['DR2T_D_TOTAL'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Dairy_EA_d2'] = Dairy_EA_d2

# Total Protein
Total_protein_EA_d1 = ((modified_df_hei['Total_Protein_D1'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Total_protein_EA_d1'] = Total_protein_EA_d1
Total_protein_EA_d2 = ((modified_df_hei['Total_Protein_D2'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Total_protein_EA_d2'] = Total_protein_EA_d2

# Seafood and Plant Proteins
Seafood_and_plant_EA_d1 = ((modified_df_hei['Seafood_and_plant_proteins_D1'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Seafood_and_plant_EA_d1'] = Seafood_and_plant_EA_d1
Seafood_and_plant_EA_d2 = ((modified_df_hei['Seafood_and_plant_proteins_D2'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Seafood_and_plant_EA_d2'] = Seafood_and_plant_EA_d2

# Refined Grains
Refined_grain_EA_d1 = ((modified_df_hei['DR1T_G_REFINED'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
modified_df_hei['Refined_grain_EA_d1'] = Refined_grain_EA_d1
Refined_grain_EA_d2 = ((modified_df_hei['DR2T_G_REFINED'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
modified_df_hei['Refined_grain_EA_d2'] = Refined_grain_EA_d2

# Sodium
# Sodium_EA_d1 = ((modified_df_hei['Sodium_D1'] / modified_df_hei['DR1TKCAL'])*1000).round(2)
Sodium_EA_d1 = modified_df_hei.apply(lambda row: safe_division(row['Sodium_D1'], row['DR1TKCAL']), axis=1).round(2)
modified_df_hei['Sodium_EA_d1'] = Sodium_EA_d1
# Sodium_EA_d2 = ((modified_df_hei['Sodium_D2'] / modified_df_hei['DR2TKCAL'])*1000).round(2)
Sodium_EA_d2 = modified_df_hei.apply(lambda row: safe_division(row['Sodium_D2'], row['DR2TKCAL']), axis=1).round(2)
modified_df_hei['Sodium_EA_d2'] = Sodium_EA_d2

# Added Sugars
Added_sugar_percent_d1 = (((modified_df_hei['DR1T_ADD_SUGARS']*16) / modified_df_hei['DR1TKCAL'])*100).round(2) # Refer to NCI (link in the preceding code) (Each tsp eq. of added sugar yields 16 kcal energy)
modified_df_hei['Added_sugar_percent_d1'] = Added_sugar_percent_d1
Added_sugar_percent_d2 = (((modified_df_hei['DR2T_ADD_SUGARS']*16) / modified_df_hei['DR2TKCAL'])*100).round(2)
modified_df_hei['Added_sugar_percent_d2'] = Added_sugar_percent_d2


# Saturated Fats
Saturated_fat_percent_d1 = (((modified_df_hei['DR1TSFAT']*9) / modified_df_hei['DR1TKCAL'])*100).round(2)
modified_df_hei['Saturated_fat_percent_d1'] = Saturated_fat_percent_d1
Saturated_fat_percent_d2 = (((modified_df_hei['DR2TSFAT']*9) / modified_df_hei['DR2TKCAL'])*100).round(2)
modified_df_hei['Saturated_fat_percent_d2'] = Saturated_fat_percent_d2






# ((modified_df_hei[''] / modified_df_hei['DR1TKCAL'])*1000).round(2)
# modified_df_hei[''] =





In [14]:
#%% Defining a function to calculate the mean (** it returns NaN whenever there's a missing value in either of the columns**)

def custom_mean(row,col_1,col_2):
  if pd.isnull(row[col_1]) or pd.isnull(row[col_2]):
    return np.nan
  else:
    return row[[col_1,col_2]].mean()

In [15]:
#%% Calculating the mean of 2-day intake of **energy-adjusted** HEI components

###*** The first calculation functions in a way that if there is a missing value, then the only existing one (the one with non-missing) will be output***###

# # Total Fruit
## modified_df_hei['Total_fruit_mean'] = (modified_df_hei[['Total_fruit_EA_d1', 'Total_fruit_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Total_fruit_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Total_fruit_EA_d1', 'Total_fruit_EA_d2'), axis=1).round(2)

# # Whole Fruit
## modified_df_hei['Whole_fruit_mean'] = (modified_df_hei[['Whole_fruit_EA_d1', 'Whole_fruit_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Whole_fruit_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Whole_fruit_EA_d1', 'Whole_fruit_EA_d2'), axis=1).round(2)

# # Total Vegetables
# modified_df_hei['Total_veg_mean'] = (modified_df_hei[['Total_veg_EA_d1', 'Total_veg_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Total_veg_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Total_veg_EA_d1', 'Total_veg_EA_d2'), axis=1).round(2)

# # Greens and Beans
# modified_df_hei['Green_bean_mean'] = (modified_df_hei[['Greens_Beans_EA_d1', 'Greens_Beans_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Green_bean_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Greens_Beans_EA_d1', 'Greens_Beans_EA_d2'), axis=1).round(2)

# # Whole Grains
# modified_df_hei['Whole_grains_mean'] = (modified_df_hei[['Whole_grains_EA_d1', 'Whole_grains_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Whole_grains_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Whole_grains_EA_d1', 'Whole_grains_EA_d2'), axis=1).round(2)

# # Dairy
# modified_df_hei['Dairy_mean'] = (modified_df_hei[['Dairy_EA_d1', 'Dairy_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Dairy_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Dairy_EA_d1', 'Dairy_EA_d2'), axis=1).round(2)

# # Total Protein Foods
# modified_df_hei['Total_protein_mean'] = (modified_df_hei[['Total_protein_EA_d1', 'Total_protein_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Total_protein_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Total_protein_EA_d1', 'Total_protein_EA_d2'), axis=1).round(2)

# # Seafood and Plant Proteins
# modified_df_hei['Seafood_plant_protein_mean'] = (modified_df_hei[['Seafood_and_plant_EA_d1', 'Seafood_and_plant_EA_d1']].mean(axis = 1)).round(2)
modified_df_hei['Seafood_plant_protein_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Seafood_and_plant_EA_d1', 'Seafood_and_plant_EA_d1'), axis=1).round(2)

# # Fatty Acids
# modified_df_hei['Fatty_acids_mean'] = (modified_df_hei[['Fatty_acids_D1', 'Fatty_acids_D2']].mean(axis = 1)).round(2)
modified_df_hei['Fatty_acids_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Fatty_acids_D1', 'Fatty_acids_D2'), axis=1).round(2)



# # Refined Grains
# modified_df_hei['Refined_grain_mean'] = (modified_df_hei[['Refined_grain_EA_d1', 'Refined_grain_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Refined_grain_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Refined_grain_EA_d1', 'Refined_grain_EA_d2'), axis=1).round(2)

# # Sodium
# modified_df_hei['Sodium_mean'] = (modified_df_hei[['Sodium_EA_d1', 'Sodium_EA_d2']].mean(axis = 1)).round(2)
modified_df_hei['Sodium_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Sodium_EA_d1', 'Sodium_EA_d2'), axis=1).round(2)

# # Added Sugar
# modified_df_hei['Added_sugar_mean'] = (modified_df_hei[['Added_sugar_percent_d1', 'Added_sugar_percent_d2']].mean(axis = 1)).round(2)
modified_df_hei['Added_sugar_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Added_sugar_percent_d1', 'Added_sugar_percent_d2'), axis=1).round(2)

# # Saturated Fat
# modified_df_hei['Saturated_fat_mean'] = (modified_df_hei[['Saturated_fat_percent_d1', 'Saturated_fat_percent_d2']].mean(axis = 1)).round(2)
modified_df_hei['Saturated_fat_mean'] = modified_df_hei.apply(lambda row: custom_mean(row, 'Saturated_fat_percent_d1', 'Saturated_fat_percent_d2'), axis=1).round(2)






In [16]:
print(modified_df_hei.columns)

Index(['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'RIDRETH3', 'DMDBORN4',
       'DMDEDUC2', 'DMDMARTZ', 'RIDEXPRG', 'INDFMPIR',
       ...
       'Green_bean_mean', 'Whole_grains_mean', 'Dairy_mean',
       'Total_protein_mean', 'Seafood_plant_protein_mean', 'Fatty_acids_mean',
       'Refined_grain_mean', 'Sodium_mean', 'Added_sugar_mean',
       'Saturated_fat_mean'],
      dtype='object', length=336)


In [17]:
# print((modified_df_hei['Sodium_EA_d1'] == np.inf).sum())
# print((modified_df_hei['Sodium_EA_d2'] == np.inf).sum())
# print((modified_df_hei['Sodium_mean'] == np.inf).sum())

# inf_rows = modified_df_hei['Sodium_mean'] == np.inf
# print(modified_df_hei.loc[inf_rows, ['Sodium_EA_d1', 'Sodium_EA_d2']])


In [18]:
#%% Investigating the built variables

row_num = len(df_modified_excl)

# print(df_modified_excl[''].describe())
# print(df_modified_excl[''].isnull().sum())
# print(((df_modified_excl[''].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage

# print(modified_df_hei[''].describe())
# print(modified_df_hei[''].isnull().sum())
# print(((modified_df_hei[''].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage


# df_modified_excl['Depression_total'].describe()
# df_modified_excl['Depression_total'].isnull().sum()
# df_modified_excl['Depression_total'].isnull().sum() / row_num * 100
# df_modified_excl['PA_vigorous_total'].describe()
# df_modified_excl['PA_vigorous_total'].isnull().sum() / row_num * 100            ### More that 91% missing value in vigorous PA !!! ###
# df_modified_excl['Whole_Fruit_D1'].describe()
# df_modified_excl['Whole_Fruit_D1'].isnull().sum()
# df_modified_excl['Whole_Fruit_D1'].isnull().sum() / row_num * 100   # Missing Percentage
# print(df_modified_excl['Whole_Fruit_D1'].describe())
# print(df_modified_excl['DR1T_F_TOTAL'].describe())

# print(modified_df_hei['Total_fruit_mean'].describe())
# print(modified_df_hei['Total_fruit_mean'].isnull().sum())
# print(((modified_df_hei['Total_fruit_mean'].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage

# print(modified_df_hei['Whole_fruit_mean'].describe())
# print(modified_df_hei['Whole_fruit_mean'].isnull().sum())
# print(((modified_df_hei['Whole_fruit_mean'].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage

# print(modified_df_hei['Total_veg_mean'].describe())
# print(modified_df_hei['Total_veg_mean'].isnull().sum())
# print(((modified_df_hei['Total_veg_mean'].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage

# print(modified_df_hei['Green_bean_mean'].describe())
# print(modified_df_hei['Green_bean_mean'].isnull().sum())
# print(((modified_df_hei['Green_bean_mean'].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage

# print(modified_df_hei['Whole_grains_mean'].describe())
# print(modified_df_hei['Whole_grains_mean'].isnull().sum())
# print(((modified_df_hei['Whole_grains_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Dairy_mean'].describe())
# print(modified_df_hei['Dairy_mean'].isnull().sum())
# print(((modified_df_hei['Dairy_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Total_protein_mean'].describe())
# print(modified_df_hei['Total_protein_mean'].isnull().sum())
# print(((modified_df_hei['Total_protein_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Seafood_plant_protein_mean'].describe())
# print(modified_df_hei['Seafood_plant_protein_mean'].isnull().sum())
# print(((modified_df_hei['Seafood_plant_protein_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Fatty_acids_mean'].describe())
# print(modified_df_hei['Fatty_acids_mean'].isnull().sum())
# print(((modified_df_hei['Fatty_acids_mean'].isnull().sum() / row_num) * 100).round(2))


# print(modified_df_hei['Refined_grain_mean'].describe())
# print(modified_df_hei['Refined_grain_mean'].isnull().sum())
# print(((modified_df_hei['Refined_grain_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Refined_grain_EA_d1'].describe())
# print(modified_df_hei['Refined_grain_EA_d1'].isnull().sum())
# print(((modified_df_hei['Refined_grain_EA_d1'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Refined_grain_EA_d2'].describe())
# print(modified_df_hei['Refined_grain_EA_d2'].isnull().sum())
# print(((modified_df_hei['Refined_grain_EA_d2'].isnull().sum() / row_num) * 100).round(2))



# print(modified_df_hei['Sodium_mean'].describe())
# print(modified_df_hei['Sodium_mean'].isnull().sum())
# print(((modified_df_hei['Sodium_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Sodium_D1'].describe())
# print(modified_df_hei['Sodium_D1'].isnull().sum())
# print(((modified_df_hei['Sodium_D1'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Sodium_D2'].describe())
# print(modified_df_hei['Sodium_D2'].isnull().sum())
# print(((modified_df_hei['Sodium_D2'].isnull().sum() / row_num) * 100).round(2))





# print(modified_df_hei['Added_sugar_mean'].describe())
# print(modified_df_hei['Added_sugar_mean'].isnull().sum())
# print(((modified_df_hei['Added_sugar_mean'].isnull().sum() / row_num) * 100).round(2))

# print(modified_df_hei['Saturated_fat_mean'].describe())
# print(modified_df_hei['Saturated_fat_mean'].isnull().sum())
# print(((modified_df_hei['Saturated_fat_mean'].isnull().sum() / row_num) * 100).round(2))




# print(modified_df_hei['Fatty_acids_mean'].describe())
# print(modified_df_hei['Fatty_acids_mean'].isnull().sum())
# print(((modified_df_hei['Fatty_acids_mean'].isnull().sum() / row_num) * 100).round(2))  # Missing Percentage


In [30]:
#%% Redefining the food group information to be used in HEI-2020
HEI_dataset = {
    'Total Fruits': modified_df_hei['Total_fruit_mean'],
    'Whole Fruits': modified_df_hei['Whole_fruit_mean'],
    'Total Vegetables': modified_df_hei['Total_veg_mean'],
    'Greens and Beans': modified_df_hei['Green_bean_mean'],
    'Whole Grains': modified_df_hei['Whole_grains_mean'],
    'Dairy': modified_df_hei['Dairy_mean'],
    'Total Protein Foods': modified_df_hei['Total_protein_mean'],
    'Seafood and Plant Proteins': modified_df_hei['Seafood_plant_protein_mean'],
    'Fatty Acids Ratio (PUFAs + MUFAs)/SFAs ': modified_df_hei['Fatty_acids_mean'],
    "Refined Grains" : modified_df_hei['Refined_grain_mean'],
    "Sodium" : modified_df_hei['Sodium_mean'],
    "Added Sugars" : modified_df_hei['Added_sugar_mean'],
    "Saturated Fats" : modified_df_hei['Saturated_fat_mean']
}

HEI_dataset_df = pd.DataFrame(HEI_dataset)





In [33]:
# #%% HEI-2020 calculation (** Missing values are regarded as zeros** (i.e Testing the function)**)

import pandas as pd
import numpy as np

def calculate_hei_2020(df):
    # Define the standards for maximum and minimum scores
    standards = {
        'Total Fruits': {'max': 0.8, 'min': 0, 'max_points': 5, 'adequacy': True},
        'Whole Fruits': {'max': 0.4, 'min': 0, 'max_points': 5, 'adequacy': True},
        'Total Vegetables': {'max': 1.1, 'min': 0, 'max_points': 5, 'adequacy': True},
        'Greens and Beans': {'max': 0.2, 'min': 0, 'max_points': 5, 'adequacy': True},
        'Whole Grains': {'max': 1.5, 'min': 0, 'max_points': 10, 'adequacy': True},
        'Dairy': {'max': 1.3, 'min': 0, 'max_points': 10, 'adequacy': True},
        'Total Protein Foods': {'max': 2.5, 'min': 0, 'max_points': 5, 'adequacy': True},
        'Seafood and Plant Proteins': {'max': 0.8, 'min': 0, 'max_points': 5, 'adequacy': True},
        'Fatty Acids Ratio (PUFAs + MUFAs)/SFAs ': {'max' :2.5 , 'min' :1.2 , 'max_points' :10 , 'adequacy' : True},

        # Moderation components

        "Refined Grains" : {"max" :1.8 , "min" :4.3 , "max_points" :10 , "adequacy" : False },
        "Sodium" : {"max" :1.1 , "min" :2.0 , "max_points" :10 , "adequacy" : False },
        "Added Sugars" : {"max" :6.5 , "min" :26 , "max_points" :10 , "adequacy" : False },
        "Saturated Fats" : {"max" :8 , "min" :16 , "max_points" :10 , "adequacy" : False }
    }

    # Initialize an empty DataFrame to store the scores
    scores = pd.DataFrame(index=df.index)

    # Calculate scores for each component
    for component in standards.keys():
        max_val = standards[component]['max']
        min_val = standards[component]['min']
        max_points = standards[component]['max_points']
        adequacy = standards[component]['adequacy']

        if adequacy:
            # For adequacy components: higher intake -> higher score
            scores[component] = ((df[component] - min_val) / (max_val - min_val)) * max_points
        else:
            # For moderation components: higher intake -> lower score
            scores[component] = max_points - ((df[component] - max_val) / (min_val - max_val)) * max_points

        # Ensure that scores are within the range [0,max_points]
        scores[component] = np.clip(scores[component], 0, max_points)

    # Sum up the scores to get the final HEI-2015 score
    scores['HEI-2020'] = scores.fillna(0).sum(axis=1)

    return scores

In [36]:
HEI_2020_test = calculate_hei_2020(HEI_dataset_df).round(2)
HEI_2020_test
print(HEI_2020_test.describe())

       Total Fruits  Whole Fruits  Total Vegetables  Greens and Beans  \
count   6539.000000   6539.000000       6539.000000       6539.000000   
mean       2.258960      2.421511          3.129621          1.987192   
std        1.988972      2.199148          1.532977          2.152909   
min        0.000000      0.000000          0.000000          0.000000   
25%        0.120000      0.000000          1.860000          0.000000   
50%        1.870000      2.250000          3.140000          1.000000   
75%        4.620000      5.000000          4.910000          5.000000   
max        5.000000      5.000000          5.000000          5.000000   

       Whole Grains        Dairy  Total Protein Foods  \
count   6539.000000  6539.000000          6539.000000   
mean       2.649345     4.536967             4.521545   
std        3.159884     2.970263             0.949546   
min        0.000000     0.000000             0.000000   
25%        0.000000     2.150000             4.480000   


In [21]:
# #%% HEI-2020 calculation (**After imputation**)

# import pandas as pd
# import numpy as np

# def calculate_hei_2020(df):
#     # Define the standards for maximum and minimum scores
#     standards = {
#         'Total Fruits': {'max': 0.8, 'min': 0, 'max_points': 5, 'adequacy': True},
#         'Whole Fruits': {'max': 0.4, 'min': 0, 'max_points': 5, 'adequacy': True},
#         'Total Vegetables': {'max': 1.1, 'min': 0, 'max_points': 5, 'adequacy': True},
#         'Greens and Beans': {'max': 0.2, 'min': 0, 'max_points': 5, 'adequacy': True},
#         'Whole Grains': {'max': 1.5, 'min': 0, 'max_points': 10, 'adequacy': True},
#         'Dairy': {'max': 1.3, 'min': 0, 'max_points': 10, 'adequacy': True},
#         'Total Protein Foods': {'max': 2.5, 'min': 0, 'max_points': 5, 'adequacy': True},
#         'Seafood and Plant Proteins': {'max': 0.8, 'min': 0, 'max_points': 5, 'adequacy': True},
#         'Fatty Acids Ratio (PUFAs + MUFAs)/SFAs ': {'max' :2.5 , 'min' :1.2 , 'max_points' :10 , 'adequacy' : True},

#         # Moderation components

#         "Refined Grains" : {"max" :1.8 , "min" :4.3 , "max_points" :10 , "adequacy" : False },
#         "Sodium" : {"max" :1.1 , "min" :2.0 , "max_points" :10 , "adequacy" : False },
#         "Added Sugars" : {"max" :6.5 , "min" :26 , "max_points" :10 , "adequacy" : False },
#         "Saturated Fats" : {"max" :8 , "min" :16 , "max_points" :10 , "adequacy" : False }
#     }

#     # Initialize an empty DataFrame to store the scores
#     scores = pd.DataFrame(index=df.index)

#     # Calculate scores for each component
#     for component in standards.keys():
#         max_val = standards[component]['max']
#         min_val = standards[component]['min']
#         max_points = standards[component]['max_points']
#         adequacy = standards[component]['adequacy']

#         if adequacy:
#             # For adequacy components: higher intake -> higher score
#             scores[component] = ((df[component] - min_val) / (max_val - min_val)) * max_points
#         else:
#             # For moderation components: higher intake -> lower score
#             scores[component] = max_points - ((df[component] - max_val) / (min_val - max_val)) * max_points

#         # Ensure that scores are within the range [0,max_points]
#         scores[component] = np.clip(scores[component], 0, max_points)

#     # Sum up the scores to get the final HEI-2015 score
#     scores['HEI-2020'] = scores.sum(axis=1)

#     return scores

In [ ]:
#%% Deleting the variables used to estimate HEI-2020 score



In [22]:
#%% Saving the files (Separately as for one-day and two-day dietary data)
# save_dataset_excel(df_modified_excl, '/content/drive/MyDrive/2017-prepand NHANES datasets/Modified Dataset(1-day).xlsx')
# save_dataset_excel(df_modified_excl, '/content/drive/MyDrive/2017-prepand NHANES datasets/Modified Dataset(2-days).xlsx')
# save_dataset_excel(modified_df_hei, '/content/drive/MyDrive/2017-prepand NHANES datasets/Modified Dataset (HEI_comp)(2-days).xlsx')

In [23]:
from tables import indexes
# Defining the % of NaNs in the dataset

# missing_value_Prc = (df_modified_excl.isnull().mean()*100).round(2)
# missing_value_Prc
# missing_data = pd.DataFrame({'Missing_percentage': missing_value_Prc})
# missing_data

In [24]:
# Counting the number of attributes with high missing
threshold = 0.5
missing_proportion = df_modified_excl.isnull().mean()
high_missing_attributes = missing_proportion[missing_proportion >= threshold].index.tolist()
high_missing_attributes

# num_high_missing_attributes = len(columns_to_remove)
# num_high_missing_attributes

['RIDEXPRG',
 'BMIWT',
 'BMIHT',
 'BMDBMIC',
 'BMIWAIST',
 'BMIHIP',
 'CBQ541',
 'CBQ586',
 'CBQ835',
 'CBQ840',
 'CBQ850',
 'CBQ855',
 'CBQ865',
 'CBQ870',
 'CBQ885',
 'CBQ895',
 'CBQ900',
 'CBQ695B',
 'CBQ695C',
 'DBQ010',
 'DBD030',
 'CBQ611',
 'ECD010',
 'ECD070A',
 'ECD070B',
 'ECQ080',
 'ECQ090',
 'WHQ030E',
 'MCQ080E',
 'MCQ170M',
 'MCQ170L',
 'PAQ610',
 'PAD615',
 'PAQ625',
 'PAD630',
 'PAQ640',
 'PAD645',
 'PAQ655',
 'PAD660',
 'PAQ670',
 'PAD675',
 'SMQ040',
 'SMQ050Q',
 'WHQ060',
 'WHD080A',
 'WHD080B',
 'WHD080C',
 'WHD080D',
 'WHD080E',
 'WHD080F',
 'WHD080G',
 'WHD080H',
 'WHD080I',
 'WHD080J',
 'WHD080K',
 'WHD080M',
 'WHD080N',
 'WHD080O',
 'WHD080P',
 'WHD080Q',
 'WHD080R',
 'WHD080S',
 'WHD080T',
 'WHD080U',
 'WHD080L',
 'WHD130',
 'WHQ030M',
 'WHQ500',
 'WHQ520',
 'PA_vigorous_total']

In [25]:
# Defininng a function which drops the attribute that have >= 50% missing

def excluding_invaluable_attributes (df, threshold = 0.5):
  copied_df = df.copy()
  missing_proportion = df.isnull().mean()
  columns_to_remove = missing_proportion[missing_proportion >= threshold].index.tolist()
  cleaned_df = copied_df.drop(columns = columns_to_remove)
  return cleaned_df


In [26]:
# Final stage of removing the unnecessary attributes (including the attributes which may be used as outcomes in other analyses)
# modified_df_N = modified_df_1.drop('RIDEXPRG', axis =1) # Removing unnecessary attributed (Pregnancy)